In [111]:
import pandas as pd

import ast #для безопасного преобразования строки в список с помощью функции literal_eval()
import json

In [112]:
df_full = pd.read_csv("data/target_data.csv")
df_full.head(2)

C:\Users\glaz\AppData\Local\Temp\ipykernel_7712\207094577.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("data/target_data.csv")


,addOkved,address,chiefs,dateLiquid,dateOgrn,dateReg,email,fullName,inn,kpp,...,okogu,oktmo,socialMedia,opfType,reports,incomeTotal,incomeDetail,incomeRank,totalSumGrants,totalSumContract
0,"[{'code': '72.4', 'name': 'Деятельность по соз...","422430, РЕСПУБЛИКА ТАТАРСТАН, ГОРОД БУИНСК, УЛ...","[{'name': 'ХАЛИКОВ РАШИТ РИНАТОВИЧ', 'title': ...",{'$date': '2012-12-17T00:00:00.000Z'},{'$date': '2010-03-03T00:00:00.000Z'},NaN,NaN,"НЕКОММЕРЧЕСКОЕ ПАРТНЕРСТВО ""ИНФОРМАЦИОННО-КОНС...",1.614010e+09,161401001.0,...,"{'code': None, 'name': None}","{'code': None, 'name': None}","{'facebook': None, 'instagram': None, 'ok': No...",Некоммерческие партнерства,[],0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",Наличие доходов из доступных источников не уст...,0,0
1,"[{'code': '67.20.2', 'name': 'Деятельность спе...","430030, РЕСПУБЛИКА МОРДОВИЯ, ГОРОД САРАНСК, УЛ...","[{'name': 'ПУЗАНОВ ВЛАДИМИР ВЛАДИМИРОВИЧ', 'ti...",{'$date': '2014-11-10T00:00:00.000Z'},{'$date': '2010-05-11T00:00:00.000Z'},NaN,NaN,"НЕКОММЕРЧЕСКОЕ ПАРТНЕРСТВО ""МЕЖРЕГИОНАЛЬНЫЙ НА...",1.327011e+09,132701001.0,...,"{'code': None, 'name': None}","{'code': None, 'name': None}","{'facebook': None, 'instagram': None, 'ok': No...",Некоммерческие партнерства,"[{'type': 'Заявление', 'reportId': '19445801',...",0,"{'grants': {'totalCount': 0, 'totalSum': 0}, '...",Наличие доходов из доступных источников не уст...,0,0


In [113]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752072 entries, 0 to 752071
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   addOkved            752072 non-null  object 
 1   address             752072 non-null  object 
 2   chiefs              752072 non-null  object 
 3   dateLiquid          406460 non-null  object 
 4   dateOgrn            752072 non-null  object 
 5   dateReg             321581 non-null  object 
 6   email               11086 non-null   object 
 7   fullName            752072 non-null  object 
 8   inn                 749468 non-null  float64
 9   kpp                 748411 non-null  float64
 10  mainOkved           752072 non-null  object 
 11  ogrn                752072 non-null  int64  
 12  opf                 752072 non-null  object 
 13  predecessors        752072 non-null  object 
 14  shortName           602778 non-null  object 
 15  statusInfo          752072 non-nul

Перечень столбцов/признаков, значение которых объективно не могут влияют на поставленную задачу, на целевой признак:
1. chiefs: руководители организации (не будем рассматривать вариант влияния субъекта на целевой признак, иными словами коррупцию) 
2. address: юридический адрес (регион можно определить с отдельного признака)
3. email: email организации
4. fullName: полное наименование организации
5. shortName: сокращенное наименование организации
6. inn
7. kpp
8. regionName: название региона организации
9. statusDetail
10. minjustStatus: статус НКО в системе Минюста РФ - УНРО (много пропусков, есть аналог egrulStatus)
11. minjustRegNum: реестровый номер НКО в системе Минюста РФ - УНРО
12. hasRegionalSupport: региональная поддержка в качестве СО НКО (все ячейки не заполнены)
13. okato: сведения об ОКАТО
14. oktmo: сведения об ОКТМО
15. okpo: ОКПО
16. dateReg: дата регистрации организаций (имеет более полный аналог признака - originDate)
17. dateOgrn: дата присвоения ОГРН (имеет более полный аналог признака - originDate)
18. incomeDetail: детализация сведений о доходах (имеет более полный аналог в признаках incomeTotal, totalSumGrants, totalSumContract)
19. incomeRank: уровень дохода (имеет более полный аналог в признаках incomeTotal, totalSumGrants, totalSumContract)
20. website_punycode: домен вебсайта организации (имеет более полный аналог в признаке website)
21. reports: отчеты организации, поданные в Минюст РФ (утечка целевой переменной)
22. socialMedia: сведения об учетных записях организации в соцсетях (количество заполненных строк - 6)

In [114]:
#Удаление столбцов/признаков, значение которых объективно не могут влияют на поставленную задачу, на целевой признак
df_full = df_full.drop(['chiefs', 'address', 'email', 'fullName', 'shortName', 'inn', 'kpp', 'regionName',
                        'statusDetail','minjustStatus','minjustRegNum','hasRegionalSupport','okato','oktmo',
                        'okpo','dateReg','dateOgrn','incomeDetail','incomeRank', 'website_punycode', 'reports','socialMedia'], axis=1)
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752072 entries, 0 to 752071
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   addOkved          752072 non-null  object
 1   dateLiquid        406460 non-null  object
 2   mainOkved         752072 non-null  object
 3   ogrn              752072 non-null  int64 
 4   opf               752072 non-null  object
 5   predecessors      752072 non-null  object
 6   statusInfo        752072 non-null  object
 7   successors        752072 non-null  object
 8   egrulStatus       752072 non-null  object
 9   foundersConfig    271348 non-null  object
 10  regionCode        752072 non-null  int64 
 11  originDate        752072 non-null  object
 12  minjustForm       465075 non-null  object
 13  website           5287 non-null    object
 14  okfs              752072 non-null  object
 15  okogu             752072 non-null  object
 16  opfType           752072 non-null  obj

Обработка кодов:
1. mainOkved: сведения об основном ОКВЭД - выводим только код
2. addOkved: дополнительный ОКВЭД - наличие дополнительных ОКВЭД: 1 - есть, 0 - нет
4. okfs: сведения об okfs - выводим только код
5. okogu: сведения об okogu - выводим только код
7. opf: сведения об opf - выводим только код

In [115]:
#Обработка кодов

#Функция для извлечения значений 'code' из словаря признака
def code_dict(df_dict,del_symbol,type_ret):
    #аргумент df_dict: значение признака, представляет собой словарь; 
    #                             у каждого словаря есть ключи: code, name.
    #аргумент del_symbol: количество первых символов, которые нужно удалить
    #аргумент type_ret: тип данных функции на выходе
    #return: код признака    
    df_dict = df_dict.replace("'", '') #Удаление апострофов из значений в столбце
    df_dict = df_dict[del_symbol:] #Удаление первых символов
    df_dict = df_dict.split(',')[0] #Удаление всех символов после запятой включительно
    if type_ret == 'int':
        if df_dict == 'None':
            return int(0)
        else:
            return int(df_dict)
    else:
        return df_dict

# Функция для извлечения значений 'code' из списка словарей признака
def code_list(df_list):
    #аргумент: значение признака, представляет собой список, 
    #                             элементы которого словари; 
    #                             у каждого словаря есть ключи: code, name, version.
    #return: 1 - список не пустой, 0 - список пустой
    addOkved_list = ast.literal_eval(df_list)
    if len(addOkved_list) > 0:
        return 1
    else:
        return 0

#Обработка mainOkved (сведения об основном ОКВЭД): выводим только код
df_full['mainOkved'] = df_full['mainOkved'].apply(lambda x: code_dict(x,7,'str'))
#Обработка addOkved (дополнительный ОКВЭД): наличие дополнительных ОКВЭД: 1 - есть, 0 - нет
df_full['addOkved'] = df_full['addOkved'].apply(code_list)
#Обработка okfs (расшифровка кода ОКФС): выводим только код
df_full['okfs'] = df_full['okfs'].apply(lambda x: code_dict(x,7,'int'))
#Обработка okogu (расшифровка кода ОКОГУ): выводим только код
df_full['okogu'] = df_full['okogu'].apply(lambda x: code_dict(x,7,'int'))
#Обработка opf (расшифровка кода ОПФ): выводим только код
df_full['opf'] = df_full['opf'].apply(lambda x: code_dict(x,7,'int'))
#Обработка statusInfo: выводим только код
df_full['statusInfo'] = df_full['statusInfo'].apply(lambda x: code_dict(x,8,'int'))

df_full[['mainOkved','addOkved','okfs','okogu','opf','statusInfo']]


,mainOkved,addOkved,okfs,okogu,opf,statusInfo
0,74.1,1,0,0,96,0
1,75.25.1,1,0,0,96,0
2,None,0,0,0,93,0
3,None,0,0,0,84,0
4,None,0,0,0,85,0
...,...,...,...,...,...,...
752067,64.99,1,16,4210014,70400,0
752068,69,1,0,0,20614,0
752069,None,0,0,0,83,0
752070,94.99,1,53,4220003,70403,0


Обработка дат
1. originDate: дата регистрации организаций
2. dateLiquid: дата ликвидации юрлица

In [116]:
#Обработка дат

#Для действующих организаций значение признака dateLiquid поставим в '0'
df_full['dateLiquid'] = df_full['dateLiquid'].fillna('0')

#Функция для извлечения года из признака
def date_dict(df_date):
    #аргумент: значение признака, которое представляет собой дату следующего вида - {'$date': '2012-12-17T00:00:00.000Z'}
    #return: год (тип float)    
    if df_date != '0':
        df_date = df_date.replace("'", '') #Удаление апострофов из значений в столбце
        df_date = df_date[7:] #Удаление первых символов
        df_date = df_date.split('-')[0] #Удаление всех символов после запятой включительно
        return int(df_date)
    else:
        return int(df_date)

df_full['originDate'] = df_full['originDate'].apply(lambda x: date_dict(x))
df_full['dateLiquid'] = df_full['dateLiquid'].apply(lambda x: date_dict(x))

df_full[['originDate','dateLiquid']]

,originDate,dateLiquid
0,2010,2012
1,2010,2014
2,1991,2008
3,1997,2009
4,1990,2008
...,...,...
752067,2013,0
752068,2013,2018
752069,2001,2007
752070,2012,0


In [117]:
df_full.head(2)

,addOkved,dateLiquid,mainOkved,ogrn,opf,predecessors,statusInfo,successors,egrulStatus,foundersConfig,regionCode,originDate,minjustForm,website,okfs,okogu,opfType,incomeTotal,totalSumGrants,totalSumContract
0,1,2012,74.1,1101600000480,96,[],0,[],Ликвидирована,Только юрлица,16,2010,Некоммерческое партнерство,NaN,0,0,Некоммерческие партнерства,0,0,0
1,1,2014,75.25.1,1101300000669,96,[],0,[],Ликвидирована,Только юрлица,13,2010,Некоммерческое партнерство,NaN,0,0,Некоммерческие партнерства,0,0,0


Перевод значений признаков в бинарный тип:
1. egrulStatus: статус организации на основании данных ЕГРЮЛ; 1 - Действует, 0 - Ликвидирована
2. website: наличие сайта; 1 - имеется, 0 - нет 
3. predecessors: предшественники; 1 - имеется, 0 - нет
4. successors: предшественники; 1 - имеется, 0 - нет

In [118]:
#Перевод значений признаков в бинарный тип

#egrulStatus: статус организации на основании данных ЕГРЮЛ; 1 - Действует, 0 - Ликвидирована
df_full['egrulStatus'] = df_full['egrulStatus'].apply(lambda x: 1 if x == 'Действует' else 0)

# Для признака website замена незаполненных значений на 0 и заполненных значений на 1
df_full['website'] = df_full['website'].fillna(0).apply(lambda x: 1 if x != 0 else x)

#predecessors: предшественники; 1 - имеется, 0 - нет
df_full['predecessors'] = df_full['predecessors'].apply(lambda x: 0 if x == '[]' else 1)

#successors: предшественники; 1 - имеется, 0 - нет
df_full['successors'] = df_full['successors'].apply(lambda x: 0 if x == '[]' else 1)

df_full[['egrulStatus','website','predecessors','successors']]

,egrulStatus,website,predecessors,successors
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
...,...,...,...,...
752067,1,0,0,0
752068,0,0,0,0
752069,0,0,0,0
752070,1,0,0,0


In [119]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752072 entries, 0 to 752071
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   addOkved          752072 non-null  int64 
 1   dateLiquid        752072 non-null  int64 
 2   mainOkved         752072 non-null  object
 3   ogrn              752072 non-null  int64 
 4   opf               752072 non-null  int64 
 5   predecessors      752072 non-null  int64 
 6   statusInfo        752072 non-null  int64 
 7   successors        752072 non-null  int64 
 8   egrulStatus       752072 non-null  int64 
 9   foundersConfig    271348 non-null  object
 10  regionCode        752072 non-null  int64 
 11  originDate        752072 non-null  int64 
 12  minjustForm       465075 non-null  object
 13  website           752072 non-null  int64 
 14  okfs              752072 non-null  int64 
 15  okogu             752072 non-null  int64 
 16  opfType           752072 non-null  obj